In [442]:
# Load the packages
import pandas as pd
import altair as alt

### 1. Load the Data

In this step, we load the datasets into Python using the `pandas` library to demonstrate that the dataset can be loaded into Python.
We read the `players.csv` and `sessions.csv` files, which contain essential details about player characteristics and session activities, respectively.

The code used for this step accomplishes the following:
- It imports the data from the relative file paths into pandas DataFrames.
- It displays the first three rows of each dataset to provide a quick overview of the data structure and contents, allowing us to verify that the datasets have been loaded correctly.

In [443]:
# 1. Load the datasets
players = pd.read_csv('data\players.csv')
sessions = pd.read_csv('data\sessions.csv')

### 2. Wrangle and Clean the Data to the Format Necessary for the Planned Analysis

#### Data Wrangling Steps:
1. Convert All Data into Proper Data Type

2. Verify the Integrity of the Data

3. Handling Missing Data and Dropping Unnecessary Columns 

4. Combine two dataframe

#### 2-1. Convert All Data into Proper Data Type: 

Proper data types are essential for efficient data processing and accurate analysis. This step ensures that each column in our datasets is stored in the most appropriate format, reflecting the nature of the data and optimizing for both memory usage and processing speed. We will convert date columns to datetime objects and other relevant columns to categorical or numerical types based on their content and role in our analysis.


In [444]:
## check the origin data type
print(players.dtypes)
print()
print(sessions.dtypes)

experience           object
subscribe              bool
hashedEmail          object
played_hours        float64
name                 object
gender               object
age                   int64
individualId        float64
organizationName    float64
dtype: object

hashedEmail             object
start_time              object
end_time                object
original_start_time    float64
original_end_time      float64
dtype: object


In [445]:
# Converting data types in 'players'
players['experience'] = players['experience'].astype('category')
players['gender'] = players['gender'].astype('category')

# Converting data types in 'sessions'
sessions['start_time'] = pd.to_datetime(sessions['start_time'],dayfirst=True)
sessions['end_time'] = pd.to_datetime(sessions['end_time'],dayfirst=True)

## check the origin data type again
print(players.dtypes)
print()
print(sessions.dtypes)

experience          category
subscribe               bool
hashedEmail           object
played_hours         float64
name                  object
gender              category
age                    int64
individualId         float64
organizationName     float64
dtype: object

hashedEmail                    object
start_time             datetime64[ns]
end_time               datetime64[ns]
original_start_time           float64
original_end_time             float64
dtype: object


By converting `experience` and `gender` in the `players` dataset to categorical types, we enhance the efficiency of our data storage and simplify the analysis involving these variables. 

By converting `start_time` and `end_time` from the `sessions` dataset to datetime, we enhance accurate and efficient time-based calculations. This adjustment ensures that our data handling is robust and that our analyses will be based on correctly formatted data, enabling precise and reliable results.


#### 2-2. Verifying the Integrity of the Data

Ensuring data integrity is a critical step before having any other data analysis. We need to check for missing values across the datasets. Missing data may significantly impact the process of the analysis, leading to biased or incorrect conclusions if not properly addressed. By identifying missing values before further analysis, we can decide on appropriate strategies for handling them, such as imputation or removal, ensuring a robust dataset for subsequent analyses.

In [446]:
# 2. verifying the integrity of the data

## Check for missing values in each column
missing_data_counts_players = players.isnull().sum()
missing_data_counts_sessions = sessions.isnull().sum()
print(missing_data_counts_players)
print()
print(missing_data_counts_sessions)

experience            0
subscribe             0
hashedEmail           0
played_hours          0
name                  0
gender                0
age                   0
individualId        196
organizationName    196
dtype: int64

hashedEmail            0
start_time             0
end_time               2
original_start_time    0
original_end_time      2
dtype: int64


By checking for missing values in each column of the `players` dataset, we can see that most columns in the `players` dataset are complete except for `individualId` and `organizationName`, which are entirely missing. This indicates that these columns may not provide any useful information for our analysis, as they contain no data at all. So we will drop them in the following wrangling.

By checking for missing values in each column of the `players` dataset, we can see that the `end_time` and `original_end_time` columns each have 2 missing entries, suggesting minor issues with data recording for these specific sessions. Considering the number of missing data is small, we may fill missing `end_time` with the start_time plus the average session duration. The `original_end_time` is not needed for our future analysis and will be drop in following steps, so we can ignore it.


#### 2-3. Handling Missing Data and Dropping Unnecessary Columns

In the `players` DataFrame, we can identify two completely empty columns: `individualId` and `organizationName`. Since these columns contain no useful data, we will drop them from the DataFrame.

In the `sessions` DataFrame, there is missing values in the `end_time` column. Considering the number of observation that has missing data is quite small, we can simply drop them.
Also, we will drop `original_start_time` and `original_end_time` from the `sessions` DataFrame since these columns are not necessary for our analysis.

In [447]:
# handle missing data, drop needless columns

## drop players 's empty column ['individualId', 'organizationName']
players.drop(columns=['individualId', 'organizationName'],inplace = True)

# # Calculate the average duration in seconds (if time is in datetime format)
# average_duration = (sessions['end_time'] - sessions['start_time']).mean()

'''We have two choice here, so we need to pick one'''
# # Iterate over the DataFrame rows using iterrows()
# for index, row in sessions.iterrows():
#     # Check if 'end_time' is NaN
#     if pd.isnull(row['end_time']):
#         # Impute missing 'end_time' by adding the average duration to 'start_time'
#         sessions.at[index, 'end_time'] = row['start_time'] + average_duration

## drop sessions 's empty column ['individualId', 'organizationName']
sessions.drop(columns=['original_start_time', 'original_end_time'],inplace = True)

# drop the observation that contains missing valuable
sessions = sessions.dropna(subset=['end_time'])

## Check for missing values in each column again
missing_data_counts_players = players.isnull().sum()
missing_data_counts_sessions = sessions.isnull().sum()
print(missing_data_counts_players)
print()
print(missing_data_counts_sessions)

experience      0
subscribe       0
hashedEmail     0
played_hours    0
name            0
gender          0
age             0
dtype: int64

hashedEmail    0
start_time     0
end_time       0
dtype: int64


#### 2-4. Combining DataFrames and Analyzing Session Durations

In [448]:
# Calculate session durations in hours
sessions['session_duration'] = (sessions['end_time'] - sessions['start_time']).dt.total_seconds() / 3600

# Aggregate total session time per player
total_session_time = sessions.groupby('hashedEmail')['session_duration'].sum()

# Merge this with player data
players = players.merge(total_session_time, how='left', left_on='hashedEmail', right_index=True)

# Compare the calculated total session time with 'played_hours'
players['discrepancy'] = players['played_hours'] - players['session_duration']

# Fill NaN only in specific columns to avoid issues with categorical data
players[['session_duration', 'discrepancy']] = players[['session_duration', 'discrepancy']].fillna(0)
players

# Check discrepancies
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,session_duration,discrepancy
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,33.650000,-3.350000
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,4.250000,-0.450000
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,0.083333,-0.083333
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,0.833333,-0.133333
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,0.150000,-0.050000
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,0.000000,0.000000
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,0.350000,-0.050000
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,0.083333,-0.083333
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,2.983333,-0.683333


### There may be big changes in the following code, so Remember to come back and have a look. All following codes will be general and should be robust enough to handle the possible change in dataframe players
# !!!
### btw, i really don't like matching codes with the requirements, so i just go and write all the codes, let's look back in the future to decide whether they are needed and which part should they belong.

In [449]:
# drop hashedEmail, name
players.drop(columns=['hashedEmail',"name","discrepancy"],inplace = True)
players

,experience,subscribe,played_hours,gender,age,session_duration
0,Pro,True,30.3,Male,9,33.650000
1,Veteran,True,3.8,Male,17,4.250000
2,Veteran,False,0.0,Male,17,0.083333
3,Amateur,True,0.7,Female,21,0.833333
4,Regular,True,0.1,Male,21,0.150000
...,...,...,...,...,...,...
191,Amateur,True,0.0,Female,17,0.000000
192,Veteran,False,0.3,Male,22,0.350000
193,Amateur,False,0.0,Prefer not to say,17,0.083333
194,Amateur,False,2.3,Male,17,2.983333


In [450]:
players_copy = players.loc[:]

### Use one-hot encoding to turn categorical variables into vectors

In [451]:
# Applying one-hot encoding to 'experience' and 'gender'
experience_dummies = pd.get_dummies(players['experience'], prefix='experience').astype(int)
gender_dummies = pd.get_dummies(players['gender'], prefix='gender').astype(int)

print(experience_dummies)
print(gender_dummies)

     experience_Amateur  experience_Beginner  experience_Pro  \
0                     0                    0               1   
1                     0                    0               0   
2                     0                    0               0   
3                     1                    0               0   
4                     0                    0               0   
..                  ...                  ...             ...   
191                   1                    0               0   
192                   0                    0               0   
193                   1                    0               0   
194                   1                    0               0   
195                   0                    0               1   

     experience_Regular  experience_Veteran  
0                     0                   0  
1                     0                   1  
2                     0                   1  
3                     0                   0  
4

In [452]:
# Since 'subscribe' only contains two types (True and False), we can convert it directly to an integer.
players['subscribe'] = players['subscribe'].astype(int)  # Replaces True with 1 and False with 0

# Concatenate the new columns back to the original dataframe
players = pd.concat([players, experience_dummies, gender_dummies], axis=1)

# Drop the original 'experience' and 'gender' columns as they are now encoded
players.drop(['experience', 'gender'], axis=1, inplace=True)

players

,subscribe,played_hours,age,session_duration,experience_Amateur,experience_Beginner,experience_Pro,experience_Regular,experience_Veteran,gender_Agender,gender_Female,gender_Male,gender_Non-binary,gender_Other,gender_Prefer not to say,gender_Two-Spirited
0,1,30.3,9,33.650000,0,0,1,0,0,0,0,1,0,0,0,0
1,1,3.8,17,4.250000,0,0,0,0,1,0,0,1,0,0,0,0
2,0,0.0,17,0.083333,0,0,0,0,1,0,0,1,0,0,0,0
3,1,0.7,21,0.833333,1,0,0,0,0,0,1,0,0,0,0,0
4,1,0.1,21,0.150000,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1,0.0,17,0.000000,1,0,0,0,0,0,1,0,0,0,0,0
192,0,0.3,22,0.350000,0,0,0,0,1,0,0,1,0,0,0,0
193,0,0.0,17,0.083333,1,0,0,0,0,0,0,0,0,0,1,0
194,0,2.3,17,2.983333,1,0,0,0,0,0,0,1,0,0,0,0


In [453]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor  # or RandomForestClassifier if it's a classification task
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [454]:
# Output dataframes instead of arrays
set_config(transform_output="pandas")

# Set the random seed for reproducibility
np.random.seed(42) # 42 is the answer to the “ultimate question of life, the universe, and everything,”(Just Joking)

In [458]:
# Define features and target variable
X = players.drop('played_hours', axis=1)  # Replace 'target_variable_name' with the name of your target column
y = players['played_hours']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [459]:
# Initialize the Random Forest model
rf = RandomForestRegressor(n_estimators=100)

# Fit the model on the training data
rf.fit(X_train, y_train)


RandomForestRegressor()

In [460]:
# Predict on the test data
y_pred = rf.predict(X_test)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE on Test Set:", rmse)

RMSE on Test Set: 1.171525234470006


In [461]:
# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to view them
features_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
features_df.sort_values(by='Importance', ascending=False, inplace=True)
print(features_df)

                     Feature    Importance
2           session_duration  9.841479e-01
3         experience_Amateur  5.241539e-03
9              gender_Female  4.098004e-03
1                        age  2.453650e-03
6         experience_Regular  1.248079e-03
11         gender_Non-binary  1.159765e-03
10               gender_Male  1.034300e-03
5             experience_Pro  3.585084e-04
4        experience_Beginner  1.605423e-04
8             gender_Agender  6.879207e-05
0                  subscribe  1.465360e-05
7         experience_Veteran  1.413971e-05
13  gender_Prefer not to say  1.372787e-07
12              gender_Other  1.528665e-08
14       gender_Two-Spirited  1.361634e-08


In [462]:
# Define a parameter grid to search for best parameters for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and RMSE from GridSearch
best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)
print("Best Parameters:", best_params)
print("Best RMSE from GridSearch:", best_rmse)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'max_depth': 30, 'max_features': 'log2', 'n_estimators': 100}
Best RMSE from GridSearch: 18.022926295593898


c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
36 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.p

In [1]:
p

NameError: name 'p' is not defined

### 3. Perform a Summary of the Data Set that is Relevant for Exploratory Data Analysis Related to the Planned Analysis

In [ ]:
#  Summarise Statistics for numerical variables
summary_stats = players_copy.describe()
summary_stats

,played_hours,age,session_duration
count,196.000000,196.000000,196.000000
mean,5.845918,21.280612,6.629762
std,28.357343,9.706346,31.310015
min,0.000000,8.000000,0.000000
25%,0.000000,17.000000,0.000000
50%,0.100000,19.000000,0.166667
75%,0.600000,22.000000,0.820833
max,223.100000,99.000000,244.516667


The summary statistics provide valuable insights into the distribution of `played_hours` and `age` within the `players` dataset:

**Played Hours:**
- **Mean**: 0.99 hours—indicating most sessions are short.
- **Standard Deviation**: 3.51—highlighting high variability with some players having significantly longer sessions.
- **Range**: 0 to 30.3 hours, with 25% not engaging and 75% playing less than 0.5 hours.

**Age:**
- **Mean**: The average age of players is approximately 21.38 years.
- **Standard Deviation**: The standard deviation(9.89) is almost half of the mean but does not exceed it, suggesting a relatively uniform distribution around the mean.
- **Range**: 8 to 99 years, with 75% of players under 22 years old, suggesting a predominantly young player base. The oldest player is 99 years old, indicating a broad age range. However, this could potentially be a misentry considering.

In [ ]:
# Count frequency for Explanatory Variables

# Categorical variables
experience_counts = players_copy.groupby('experience',observed=False).size().reset_index(name='count')
gender_counts = players_copy.groupby('gender',observed=False).size().reset_index(name='count')
subscribe_counts = players_copy.groupby('subscribe',observed=False).size().reset_index(name='count')

# Numerical variavble `age`
# Define bins and labels
bins = [0, 12, 18, 35, 65, 120]
labels = ['Child(0-12)', 'Teen(12-18)', 'Young Adult(18-35)', 'Adult(35-65)', 'Senior(65-120)']
# Create a new categorical variable 'age_group'
players_copy['age_group'] = pd.cut(players_copy['age'], bins = bins, labels = labels, right = False)
age_counts = players_copy.groupby('age_group',observed=False).size().reset_index(name = 'count')

print(experience_counts,"\n")
print(gender_counts,"\n")
print(subscribe_counts,"\n")
print(age_counts)

  experience  count
0    Amateur     63
1   Beginner     35
2        Pro     14
3    Regular     36
4    Veteran     48 

              gender  count
0            Agender      2
1             Female     37
2               Male    124
3         Non-binary     15
4              Other      1
5  Prefer not to say     11
6       Two-Spirited      6 

   subscribe  count
0      False     52
1       True    144 

            age_group  count
0         Child(0-12)      4
1         Teen(12-18)     83
2  Young Adult(18-35)     99
3        Adult(35-65)      8
4      Senior(65-120)      2


### 4. Create a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis

##### Histograms for continuous variables - Age and Played Hours

In [ ]:
hist_age = alt.Chart(players).mark_bar().encode(
    alt.X('age', bin = alt.Bin(maxbins = 30), title = 'Age(Years)'),
    alt.Y('count()', title = 'Frequency')
).properties(
    title = 'Histogram of Age',
    width = 360,
    height = 200
)

hist_age

alt.Chart(...)

As is depicted in the histogram, the age groups are concentrated primarily between 15 and 30, with the highest frequency in the 15 to 25 year range. This suggests that the player base is teenagers and young adults.

In [ ]:
hist_played_hours = alt.Chart(players).mark_bar().encode(
    alt.X('played_hours', bin = alt.Bin(maxbins=30), title = 'Played Hours'),
    alt.Y('count()', title = 'Frequency')
).properties(
    title = 'Histogram of Played Time',
    width = 360,
    height = 200
)

hist_played_hours

alt.Chart(...)

As is depicted in the plot, a vast majority have logged very few hours. Specifically, the highest frequency occurs at the lowest time bracket, indicating most players spend less than 10 hours total. This distribution highlights a potential issue in player engagement, as the data suggests many players do not return or play long.

##### Bar Charts for categorical variables - Experience, Gender, Subscribe

In [ ]:
bar_experience = alt.Chart(players).mark_bar().encode(
    x = alt.X('experience:N', sort=['Beginner', 'Amateur', 'Regular', 'Veteran', 'Pro'],
     axis = alt.Axis(labelAngle=0)).title("Experience"),  # Set labelAngle to 0 for horizontal labels
    y = 'count()'
).properties(
    title = 'Player Experience Levels Counts',
    width = 240,
    height = 200
)

bar_experience

alt.Chart(...)

As is depicted in the plot, the majority of players are categorized as 'Amateur', followed by 'Regular' and 'Veteran', indicating a player base with a range of experience but leaning towards newer or moderately experienced players.

In [ ]:
bar_gender = alt.Chart(players).mark_bar().encode(
    x = alt.X('gender:N',sort='-y', axis=alt.Axis(labelAngle=0)).title("Gender"),
    y = 'count()'
).properties(
    title = 'Gender Counts',
    width = 480,
    height = 200
)
bar_gender

alt.Chart(...)

As is depicted in the plot, the distribution of gender among players shows a predominant number of male players compared to other gender identities since the number of male players is over double the number of the next largest group, females.

In [ ]:
bar_subscribe = alt.Chart(players).mark_bar().encode(
    x = alt.X('subscribe:N', axis=alt.Axis(labelAngle=0)).title("Subscribe Status"),
    y = 'count()'
).properties(
    title = 'Subscription Status Counts',
    width = 100,
    height = 200
)
bar_subscribe

alt.Chart(...)

As is depicted in the plot, there is a significantly higher number of subscribed players compared to non-subscribers.

### 5. Perform the data analysis

### 6. Create a visualization of the analysis 